In [2]:
!pip install ydata-synthetic==1.1.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 93.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 128.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.7/280.7 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 87.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 139.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [39]:
from pmlb import fetch_data

from ydata_synthetic.synthesizers.regular import RegularSynthesizer
from ydata_synthetic.synthesizers import ModelParameters, TrainParameters
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

#Load data and define the data processor parameters
df = pd.read_csv('https://raw.githubusercontent.com/mahayasa/various-sampling-churn-prediction/main/data/bank_churn.csv')
df=df.drop(['RowNumber','CustomerId','Surname'],axis=1)

df_numerical_features = df.select_dtypes(exclude='object')
df_categorical_features = df.select_dtypes(include='object')

num_cols = list(df_numerical_features)
cat_cols = list(df_categorical_features)

df['Geography']=le.fit_transform(df['Geography'])
df['Gender']=le.fit_transform(df['Gender'])

In [48]:
train_data = df.loc[ df['Exited']==1 ].copy()

In [50]:


#Defining the training parameters
noise_dim = 128
dim = 128
batch_size = 500

log_step = 100
epochs = 100
learning_rate = [5e-4, 3e-3]
beta_1 = 0.5
beta_2 = 0.9
models_dir = '../cache'

gan_args = ModelParameters(batch_size=batch_size,
                           lr=learning_rate,
                           betas=(beta_1, beta_2),
                           noise_dim=noise_dim,
                           layers_dim=dim)

train_args = TrainParameters(epochs=epochs,
                             sample_interval=log_step)

synth = RegularSynthesizer(modelname='wgangp', model_parameters=gan_args, n_critic=2)
synth.fit(train_data, train_args, num_cols, cat_cols)

synth.save('adult_wgangp_model.pkl')



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
  1%|          | 1/100 [00:02<03:19,  2.02s/it]

Epoch: 0 | disc_loss: 0.6016745567321777 | gen_loss: 0.010624761693179607


  2%|▏         | 2/100 [00:02<01:44,  1.07s/it]

Epoch: 1 | disc_loss: 0.26119938492774963 | gen_loss: 0.003991437144577503


  3%|▎         | 3/100 [00:02<01:14,  1.31it/s]

...

Epoch: 97 | disc_loss: 0.08547487109899521 | gen_loss: 0.017975343391299248


 99%|█████████▉| 99/100 [00:51<00:00,  1.95it/s]

Epoch: 98 | disc_loss: 0.13101603090763092 | gen_loss: 0.02471214160323143


100%|██████████| 100/100 [00:51<00:00,  1.93it/s]

Epoch: 99 | disc_loss: 0.022993089631199837 | gen_loss: 0.05504893884062767


In [51]:
#########################################################
#    Loading and sampling from a trained synthesizer    #
#########################################################
synth = RegularSynthesizer.load('adult_wgangp_model.pkl')
synth_data = synth.sample(5800)

Synthetic data generation: 100%|██████████| 12/12 [00:00<00:00, 137.83it/s]


In [52]:
synth_data[synth_data['Exited']==1]

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
244,499,1,0,25,1,73900.757812,0,0,0,86520.289062,1
274,539,1,0,24,1,89923.765625,0,0,0,105815.906250,1
521,543,1,0,24,0,101330.929688,0,0,0,90418.171875,1
1460,513,1,0,20,0,59772.421875,0,0,0,85102.328125,1
1928,520,2,0,24,0,73377.328125,0,0,0,80567.945312,1
2827,570,1,0,29,2,120777.703125,0,0,0,134099.046875,1
2904,536,1,0,24,0,75221.671875,0,0,0,72454.390625,1
4189,511,1,0,27,1,82623.125000,0,0,0,98824.750000,1
4357,519,1,0,25,1,59835.296875,0,0,0,86082.867188,1
4428,482,1,0,18,0,67794.453125,0,0,0,71577.125000,1


In [53]:
#concat original data and gan data
data_concat = pd.concat([df, synth_data])
# combine data churn and not churn
data=pd.concat([df, data_concat])

In [54]:
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.000000,1,1,1,101348.880000,1
1,608,2,0,41,1,83807.860000,1,0,1,112542.580000,0
2,502,0,0,42,8,159660.800000,3,1,0,113931.570000,1
3,699,0,0,39,1,0.000000,2,0,0,93826.630000,0
4,850,2,0,43,2,125510.820000,1,1,1,79084.100000,0
...,...,...,...,...,...,...,...,...,...,...,...
5995,507,2,0,24,1,72425.148438,0,0,0,83073.773438,0
5996,488,1,0,25,0,84840.171875,0,0,0,88450.250000,0
5997,486,1,0,26,0,58002.167969,0,0,0,66475.312500,0
5998,543,1,0,35,0,73414.656250,0,0,0,84242.789062,0


In [67]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jun  1 17:31:40 2020

@author: manav

Modifed on 23 AUG 2022

by mahayasa adiputra
"""

import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
import sklearn.metrics as mt
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import NeighbourhoodCleaningRule
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from numpy import mean
from numpy import std
from sklearn.metrics import make_scorer
from imblearn.metrics import specificity_score
import math
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
import time

start1=time.time()

X=data.drop(['Exited'],axis=1)
y=data["Exited"]

enn = EditedNearestNeighbours(n_neighbors=3)
X, y = enn.fit_resample(X, y)
#ncr = NeighbourhoodCleaningRule(n_neighbors=5, kind_sel='all')
#X, y = ncr.fit_resample(X, y)
#tomek_links = TomekLinks()
#X, y = tomek_links.fit_resample(X, y)

#kfold cross validation
cv = KFold(n_splits=5, random_state=1, shuffle=True)
end1 = time.time()
print("The time of execution of preprocess:",
      (end1-start1), "s")

The time of execution of preprocess: 0.4423220157623291 s


In [56]:
data[data['Exited']==1]

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.000000,1,1,1,101348.880000,1
2,502,0,0,42,8,159660.800000,3,1,0,113931.570000,1
5,645,2,1,44,8,113755.780000,2,1,0,149756.710000,1
7,376,1,0,29,4,115046.740000,4,1,0,119346.880000,1
16,653,1,1,58,1,132602.880000,1,1,0,5097.670000,1
...,...,...,...,...,...,...,...,...,...,...,...
4863,533,1,0,23,0,97896.109375,0,0,0,113225.031250,1
4912,549,1,0,32,1,68898.929688,0,0,0,93836.687500,1
5091,507,1,0,19,0,66092.390625,0,0,0,74051.906250,1
5293,545,1,0,21,0,82269.906250,0,0,0,102800.531250,1


In [68]:
startknn=time.time()
classifier=KNeighborsClassifier()
score=cross_val_score(classifier, X, y, scoring='f1', cv=cv, n_jobs=-1)
print('===============KNN Performance====================')

print('F1 score: %.3f' % (mean(score)))
print('STD F1 Score: %.3f' % (std(score)))

rc=cross_val_score(classifier, X, y, scoring='recall', cv=cv, n_jobs=-1)
print('Recall: %.3f' % (mean(rc)))


# Define the specificity scorer
scorer = make_scorer(specificity_score)

# Calculate the cross-validated specificity score
sp = cross_val_score(classifier, X, y, cv=5, scoring=scorer)
print('Specitifity: %.3f' % (mean(sp)))

auc=cross_val_score(classifier, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('AUC ROC: %.3f' % (mean(auc)))

gmean=rc*sp
gmean=mean(gmean)
sqrtg = math.sqrt(gmean)
print('G-Mean: %.3f' % (sqrtg))
print('======================================================')

endknn = time.time()
print("The time of execution of knn:",
      (endknn-startknn), "s")

===============KNN Performance====================
F1 score: 0.520
STD F1 Score: 0.013
Recall: 0.404
Specitifity: 0.967
AUC ROC: 0.856
G-Mean: 0.625
The time of execution of knn: 5.75894832611084 s


In [69]:
start3=time.time()

classifier = GradientBoostingClassifier()

score=cross_val_score(classifier, X, y, scoring='f1', cv=cv, n_jobs=-1)
print('===============GBM Performance====================')

print('F1 score: %.3f' % (mean(score)))
print('STD F1 Score: %.3f' % (std(score)))

rc=cross_val_score(classifier, X, y, scoring='recall', cv=cv, n_jobs=-1)
print('Recall: %.3f' % (mean(rc)))


# Define the specificity scorer
scorer = make_scorer(specificity_score)

# Calculate the cross-validated specificity score
sp = cross_val_score(classifier, X, y, cv=5, scoring=scorer)
print('Specitifity: %.3f' % (mean(sp)))

auc=cross_val_score(classifier, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('AUC ROC: %.3f' % (mean(auc)))


gmean=rc*sp
gmean=mean(gmean)
sqrtg = math.sqrt(gmean)
print('G-Mean: %.3f' % (sqrtg))
print('======================================================')

end3 = time.time()
print("The time of execution of gbm:",
      (end3-start3), "s")





===============GBM Performance====================
F1 score: 0.628
STD F1 Score: 0.022
Recall: 0.516
Specitifity: 0.968
AUC ROC: 0.911
G-Mean: 0.707
The time of execution of gbm: 40.46847319602966 s


In [70]:
startdt=time.time()
classifier=DecisionTreeClassifier()
score=cross_val_score(classifier, X, y, scoring='f1', cv=cv, n_jobs=-1)
print('===============DT Performance====================')

print('F1 score: %.3f' % (mean(score)))
print('STD F1 Score: %.3f' % (std(score)))

rc=cross_val_score(classifier, X, y, scoring='recall', cv=cv, n_jobs=-1)
print('Recall: %.3f' % (mean(rc)))


# Define the specificity scorer
scorer = make_scorer(specificity_score)

# Calculate the cross-validated specificity score
sp = cross_val_score(classifier, X, y, cv=5, scoring=scorer)
print('Specitifity: %.3f' % (mean(sp)))

auc=cross_val_score(classifier, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('AUC ROC: %.3f' % (mean(auc)))

gmean=rc*sp
gmean=mean(gmean)
sqrtg = math.sqrt(gmean)
print('G-Mean: %.3f' % (sqrtg))
print('======================================================')

enddt = time.time()
print("The time of execution of dt:",
      (enddt-startdt), "s")

===============DT Performance====================
F1 score: 0.894
STD F1 Score: 0.017
Recall: 0.899
Specitifity: 1.000
AUC ROC: 0.936
G-Mean: 0.948
The time of execution of dt: 1.6149017810821533 s


In [71]:
start5 = time.time()
classifier=GaussianNB()
score=cross_val_score(classifier, X, y, scoring='f1', cv=cv, n_jobs=-1)
print('===============NB Performance====================')

print('F1 score: %.3f' % (mean(score)))
print('STD F1 Score: %.3f' % (std(score)))

rc=cross_val_score(classifier, X, y, scoring='recall', cv=cv, n_jobs=-1)
print('Recall: %.3f' % (mean(rc)))


# Define the specificity scorer
scorer = make_scorer(specificity_score)

# Calculate the cross-validated specificity score
sp = cross_val_score(classifier, X, y, cv=5, scoring=scorer)
print('Specitifity: %.3f' % (mean(sp)))

auc=cross_val_score(classifier, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('AUC ROC: %.3f' % (mean(auc)))

gmean=rc*sp
gmean=mean(gmean)
sqrtg = math.sqrt(gmean)
print('G-Mean: %.3f' % (sqrtg))
print('======================================================')

end5 = time.time()
print("The time of execution of NB:",
      (end5-start5), "s")

===============NB Performance====================
F1 score: 0.361
STD F1 Score: 0.017
Recall: 0.293
Specitifity: 0.903
AUC ROC: 0.798
G-Mean: 0.515
The time of execution of NB: 0.45622920989990234 s


In [72]:


start2=time.time()
#Training XGBoost
#classifier = XGBClassifier(eta=0.3, max_depth = 4, gamma=0, min_child_weight=1)
classifier = XGBClassifier()

score=cross_val_score(classifier, X, y, scoring='f1', cv=cv, n_jobs=-1)
print('===============XGBoost Performance====================')

print('F1 score: %.3f' % (mean(score)))
print('STD F1 Score: %.3f' % (std(score)))

rc=cross_val_score(classifier, X, y, scoring='recall', cv=cv, n_jobs=-1)
print('Recall: %.3f' % (mean(rc)))



# Define the specificity scorer
scorer = make_scorer(specificity_score)

# Calculate the cross-validated specificity score
sp = cross_val_score(classifier, X, y, cv=5, scoring=scorer)
print('Specitifity: %.3f' % (mean(sp)))

auc=cross_val_score(classifier, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('AUC ROC: %.3f' % (mean(auc)))


gmean=rc*sp
gmean=mean(gmean)
sqrtg = math.sqrt(gmean)
print('G-Mean: %.3f' % (sqrtg))
print('======================================================')

end2 = time.time()
print("The time of execution of XGBOOST:",
      (end2-start2), "s")

===============XGBoost Performance====================
F1 score: 0.773
STD F1 Score: 0.008
Recall: 0.684
Specitifity: 0.987
AUC ROC: 0.959
G-Mean: 0.822
The time of execution of XGBOOST: 29.126839637756348 s


In [73]:
start3=time.time()

from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier()

score=cross_val_score(classifier, X, y, scoring='f1', cv=cv, n_jobs=-1)
print('===============Random Forest Performance====================')

print('F1 score: %.3f' % (mean(score)))
print('STD F1 Score: %.3f' % (std(score)))

rc=cross_val_score(classifier, X, y, scoring='recall', cv=cv, n_jobs=-1)
print('Recall: %.3f' % (mean(rc)))

from sklearn.metrics import make_scorer
from imblearn.metrics import specificity_score

# Define the specificity scorer
scorer = make_scorer(specificity_score)

# Calculate the cross-validated specificity score
sp = cross_val_score(classifier, X, y, cv=5, scoring=scorer)
print('Specitifity: %.3f' % (mean(sp)))

auc=cross_val_score(classifier, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('AUC ROC: %.3f' % (mean(auc)))

import math

gmean=rc*sp
gmean=mean(gmean)
sqrtg = math.sqrt(gmean)
print('G-Mean: %.3f' % (sqrtg))
print('======================================================')

end3 = time.time()
print("The time of execution of random forest:",
      (end3-start3), "s")

===============Random Forest Performance====================
F1 score: 0.928
STD F1 Score: 0.011
Recall: 0.895
Specitifity: 1.000
AUC ROC: 0.986
G-Mean: 0.946
The time of execution of random forest: 34.37539029121399 s
